# Neural Network

In [6]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install matplotlib

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import utils
from utils import TrainingPlot, TimeSummary, plot_training_summary
from NNKeras import NNKeras
from sklearn.model_selection import train_test_split
import inspect
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Step 1
### Data Preparation

#### Data Cleanup
-  Merge 64 size vectors by ignoring line breaks
-  Create subset of data by selecting non consecutive vectors

#### Cleaned data
L42023,0.04347826,0.04347826,0.,0.04347826,0.01086957,0.02173913,0.,0.02173913,0.,0.,0.,0.,0.,0.02173913,0.02173913,0.04347826,0.07608696,0.02173913,0.,0.0326087,0.01086957,0.,0.,0.0326087,0.,0.01086957,0.,0.0326087,0.,0.,0.,0.0326087,0.05434783,0.,0.01086957,0.02173913,0.04347826,0.,0.01086957,0.02173913,0.02173913,0.,0.,0.01086957,0.0326087,0.,0.04347826,0.0326087,0.01086957,0.01086957,0.,0.02173913,0.04347826,0.01086957,0.,0.01086957,0.,0.,0.,0.,0.04347826,0.02173913,0.,0.

## Network 1
### Create input dataset X and y where X has all vectors and y is one-hot vector

In [7]:
nn = NNKeras("/tf/dataset/dataset_full_rnd.csv")
X, y, unique_classes = nn.read_data()

In [8]:
print("Input Vector Shape")
print(X.shape)
print("Initialized Training Vector Shape")
print(y.shape)
print("Input Vector Head")
print(X[:3])
print("Training Vector Head")
print(y[:3])
print("Categories")
unique_classes

Input Vector Shape
(81068, 64)
Initialized Training Vector Shape
(81068, 31)
Input Vector Head
         1         2         3         4         5         6         7   \
0  0.036508  0.019048  0.052381  0.023810  0.003175  0.011111  0.004762   
1  0.060345  0.017241  0.000000  0.017241  0.025862  0.017241  0.000000   
2  0.015337  0.006135  0.015337  0.015337  0.000000  0.006135  0.018405   

         8         9         10  ...        55        56   57        58  \
0  0.007937  0.015873  0.007937  ...  0.001587  0.003175  0.0  0.001587   
1  0.034483  0.000000  0.008621  ...  0.000000  0.008621  0.0  0.025862   
2  0.009202  0.003067  0.012270  ...  0.024540  0.003067  0.0  0.003067   

         59   60        61        62        63        64  
0  0.014286  0.0  0.019048  0.011111  0.014286  0.028571  
1  0.000000  0.0  0.008621  0.034483  0.025862  0.034483  
2  0.006135  0.0  0.003067  0.000000  0.030675  0.012270  

[3 rows x 64 columns]
Training Vector Head
[[ True False False Fal

,0,indices
0,NC_000961,1
1,AE005672,2
2,AL450380,3
3,AL645882,4
4,AL009126,5
5,AB001339,6
6,AE014075,7
7,AE000782,8
8,AE009442,9
9,L42023,10


### Prepare Train and Test Data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
print(X_train[:3])
print(y_train[:3])

             1         2         3         4         5         6         7   \
61310  0.059006  0.015528  0.040373  0.040373  0.012422  0.003106  0.015528   
28317  0.031579  0.024561  0.014035  0.010526  0.014035  0.007018  0.017544   
48590  0.001818  0.010909  0.010909  0.003636  0.000000  0.016364  0.009091   

             8         9         10  ...        55        56        57  \
61310  0.006211  0.015528  0.006211  ...  0.003106  0.009317  0.003106   
28317  0.010526  0.000000  0.007018  ...  0.007018  0.010526  0.000000   
48590  0.000000  0.000000  0.036364  ...  0.005455  0.000000  0.001818   

             58        59        60        61        62        63        64  
61310  0.003106  0.018634  0.006211  0.006211  0.021739  0.031056  0.037267  
28317  0.000000  0.010526  0.000000  0.038596  0.010526  0.007018  0.017544  
48590  0.016364  0.030909  0.000000  0.000000  0.020000  0.018182  0.001818  

[3 rows x 64 columns]
[[False False False False False False False  True F

In [10]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import numpy as np


model = SVR(kernel="linear")
model.fit(X_train, np.argmax(y_train, axis=1))
y_pred = model.predict(X_test)




In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, np.argmax(y_test, axis=1))

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()
model.fit(X_train, np.argmax(y_train, axis=1))
y_pred = model.predict(X_test)
print('test acc', accuracy_score(y_pred, np.argmax(y_test, axis=1)))
y_pred_train = model.predict(X_train)
print('train acc', accuracy_score(y_pred_train, np.argmax(y_train, axis=1)))

test acc 0.8078820772172197
train acc 0.8074752521047276


In [13]:
y_prob_all = model.predict_proba(X)
list(y_prob_all[y])

[0.8533502144407433,
 0.8043424250038147,
 0.3066285127740533,
 0.20473939648892167,
 0.9918417669533364,
 0.9942873781563674,
 0.9999993905078743,
 0.00018587655304635147,
 0.9960794574113141,
 0.016325890979279077,
 0.8646697410080402,
 0.9899644196411276,
 0.9999600656975288,
 0.9922537488058185,
 0.9011494856094303,
 0.13133728018944124,
 0.9998188035018369,
 0.999944471791052,
 0.02165085013834678,
 0.9999761943553935,
 0.9952347502100911,
 0.9994010027528971,
 0.3194306882932444,
 0.7934040089769205,
 0.99988504498048,
 0.9999985478111811,
 0.9998903902284343,
 0.007711920538009108,
 0.9988477135065463,
 0.6405078553580968,
 0.9986280120254297,
 0.9987045902239635,
 0.18372294331372743,
 0.9257020648813554,
 1.7882074455912956e-11,
 0.719248304806497,
 0.9998105838796392,
 0.999999779662127,
 0.8278414756655972,
 0.8598025954455121,
 0.08923062865527148,
 0.4662514457977978,
 0.3425059296417812,
 0.9997819178618449,
 0.9998142305419275,
 0.9929775821230262,
 0.9129084650861109,
 

In [14]:
np.sum(y_prob_all[y]<0.01) / len(y)

0.06282380224009473

## Step 2
### Network 1
-  Create neural network with single hidden layer
-  Use **Accuracy** as metric of performance  


In [19]:
inspect.getsourcelines(nn.train)
inspect.getsourcelines(nn.base_model)

(['    def base_model(self, nodes, num_output=31, kernel_regularizer=None, layer_id=None):\n',
  '        model = Sequential()\n',
  '        for prev_node, node in zip(nodes[:-1], nodes[1:]):\n',
  '            layer_name = None\n',
  '            if layer_id is not None:\n',
  '                layer_name = "{}-in-{}-n-{}".format(layer_id, str(prev_node), str(prev_node))\n',
  "            model.add(Dense(node, activation='relu', kernel_regularizer=kernel_regularizer,\n",
  '                            input_dim=prev_node, name=layer_name))\n',
  "        model.add(Dense(num_output, activation='sigmoid', name='features'))\n",
  "        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n",
  '        return model\n'],
 38)

In [20]:
nn.train(X, y, False)

811/811 [==============================] - 0s 589us/step
Test loss: 0.07687149639672033
Test accuracy: 0.9741458293101055


>  With just single layer we are getting accuracy of 97% Epoch=10

## Step 3
### Network 2
-  Use the same model as step 1 for neurons  = 5 to 32
-  Measure accuracy at each step to find optimal number of neurons

In [21]:
inspect.getsourcelines(nn.train_network_2)

(['    def train_network_2(self, X, y):\n',
  '        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)\n',
  '        for P in range(2, 12):\n',
  '            for num_layers in range(1, 2):\n',
  '                nodes = [64] + [int(P / 2)] * num_layers\n',
  '                print(nodes)\n',
  '                model = self.base_model(nodes)\n',
  '                print(model.summary())\n',
  '                summary = model.fit(X_train, y_train, epochs=10, verbose=0)\n',
  '                score = model.evaluate(X_test, y_test)\n',
  "                print('Test loss:', score[0])\n",
  "                print('Test accuracy:', score[1])\n"],
 112)

In [18]:
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split

def base_model(nodes, num_output=31, kernel_regularizer=None, layer_id=None):
    model = Sequential()
    layer_name = None
    for prev_node, node in zip(nodes[:-1], nodes[1:]):
        layer_name = "{}-in-{}-n-{}".format(layer_id, str(prev_node), str(prev_node))
        model.add(Dense(node, activation='relu', kernel_regularizer=kernel_regularizer,
        input_dim=prev_node, name=layer_name))
    model.add(Dense(num_output, activation='sigmoid', name='features'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_network_debug(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)
    for P in range(2, 12):
        for num_layers in range(1, 2):
            nodes = [64] + [int(P / 2)] * num_layers
            print(nodes)
            model = base_model(nodes)
            print(model.summary())
            summary = model.fit(X_train, y_train, epochs=10, verbose=0)
            score = model.evaluate(X_test, y_test)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])

train_network_debug(X, y)

[64, 1]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
None-in-64-n-64 (Dense)      (None, 1)                 65        
_________________________________________________________________
features (Dense)             (None, 31)                62        
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________
None
4054/4054 [==============================] - 0s 52us/step
Test loss: 0.13744815848087757
Test accuracy: 0.9677419066429138
[64, 1]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
None-in-64-n-64 (Dense)      (None, 1)                 65        
_________________________________________________________________
features (Dense)             (None, 31)                62        
Total params: 127
Trainable params: 127
Non-trainable params: 0
__

In [22]:
nn.train_network_2(X, y)

[64, 1]
4054/4054 [==============================] - 0s 113us/step
Test loss: 0.12101917903177481
Test accuracy: 0.9696277563773062
[64, 1]
4054/4054 [==============================] - 1s 343us/step
Test loss: 0.12091939075935869
Test accuracy: 0.9695004410913088
[64, 2]
4054/4054 [==============================] - ETA:  - 1s 318us/step
Test loss: 0.10972485231850379
Test accuracy: 0.9692060279622711
[64, 2]
4054/4054 [==============================] - 1s 165us/step
Test loss: 0.10918649259566908
Test accuracy: 0.9704791691483214
[64, 3]
4054/4054 [==============================] - 1s 298us/step
Test loss: 0.08958604379010777
Test accuracy: 0.9722615741248429
[64, 3]
4054/4054 [==============================] - 2s 382us/step
Test loss: 0.08949642153534915
Test accuracy: 0.9719751129376224
[64, 4]
4054/4054 [==============================] - 1s 367us/step
Test loss: 0.09400902894340717
Test accuracy: 0.9707338043369562
[64, 4]
4054/4054 [==============================] - ETA:  - 2s 421u

>  ### Accuraccy is already at 96% in step 1, so we add regularize function to decrease accuracy and determine optimal number of neurons as 6
>  ### P=6 

## Step 4
### Network 3
-  Find optimal number of hidden layers with number of neurons as P/2
-  Again, since the accuracy is very high, we will have to take regularizer function
-  P=6, Number of neurons = P/2

In [23]:
inspect.getsourcelines(nn.train_network_3)

(['    def train_network_3(self, X, y):\n',
  '        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)\n',
  '        P = 6\n',
  '        for num_layers in range(1, 10):\n',
  '            nodes = [64] + [int(P / 2)] * num_layers\n',
  '            model = self.base_model(nodes)\n',
  '            print(model.summary())\n',
  '            summary = model.fit(X_train, y_train, epochs=10, verbose=0)\n',
  '            score = model.evaluate(X_test, y_test)\n',
  "            print('Test loss:', score[0])\n",
  "            print('Test accuracy:', score[1])\n"],
 100)

In [12]:
nn.train_network_3(X, y)

4054/4054 [==============================] - 0s 58us/step
Test loss: 0.09179627784717595
Test accuracy: 0.97132263212411
4054/4054 [==============================] - 0s 62us/step
Test loss: 0.08782961113689919
Test accuracy: 0.9706621874890189
4054/4054 [==============================] - 0s 69us/step
Test loss: 0.11377906092117734
Test accuracy: 0.9698903405672232
4054/4054 [==============================] - 0s 77us/step
Test loss: 0.12015273502499939
Test accuracy: 0.9702404533693457
4054/4054 [==============================] - 0s 82us/step
Test loss: 0.13819008715704384
Test accuracy: 0.9677419066429138
4054/4054 [==============================] - 0s 93us/step
Test loss: 0.1198273547908974
Test accuracy: 0.9702165809298714
4054/4054 [==============================] - 0s 104us/step
Test loss: 0.121142026733517
Test accuracy: 0.9700574382338315
4054/4054 [==============================] - 0s 122us/step
Test loss: 0.1381859623068474
Test accuracy: 0.9677419066429138
4054/4054 [=========

>  ### Because of high accuracy, we can select number of layers=6

## Step 5
### Network 4
-  Without any conditions, determine optimal architecture
-  We can run the training module with P=6 and Layers=6
-  Save model for validation

In [13]:
inspect.getsourcelines(nn.train_with_callback)

(['    def train_with_callback(self, X, y, model_file: str = None, generate_plots=True):\n',
  '        if model_file is None:\n',
  '            model_file = self._model\n',
  '        plot_losses = TrainingPlot()\n',
  '        time_summary = TimeSummary()\n',
  "        layer_names = ['features']\n",
  '        num_nodes = 6\n',
  '        for layer_id in range(6, 7):\n',
  '            nodes = [64, num_nodes]\n',
  '            for prev_node, node in zip(nodes[:-1], nodes[1:]):\n',
  '                layer_name = None\n',
  '                if layer_id is not None:\n',
  '                    layer_name = "{}-in-{}-n-{}".format(layer_id, str(prev_node), str(prev_node))\n',
  '                    layer_names.append(layer_name)\n',
  '\n',
  '        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=5)\n',
  "        with open(os.path.join(self._log_dir, 'metadata.tsv'), 'w') as f:\n",
  '            np.savetxt(f, y_test)\n',
  '        tensorboard

In [14]:
nn.train_with_callback(X, y, '/tf/models/model.h5')

KeyboardInterrupt: 

# Validation
### In this dataset, our neural network reaches accuracy of 96% with sample data and 98% with full dataset. Hence it is necessary to validate the accuracy of data with trained model.
### Keras allows callbacks for training visualization. Function below runs for 200 epochs with 32 neurons and saves the model

### Predict from souce data

In [ ]:
inspect.getsourcelines(nn.predict)

In [ ]:
nn.predict("/tf/dataset/ae002161.csv", unique_classes, 5, "/tf/models/model_full.h5")

In [ ]:
nn.predict("/tf/dataset/ae003852.csv", unique_classes, 5, "/tf/models/model_full.h5")

>  ### As we can see that the accuracy is as expected for the first case but for second, it is less than expected.

## Step 6
### Network 5
-  We will use same training function with each column of y as 1D matrix
-  Take average of each inidividual prediction

In [ ]:
inspect.getsourcelines(nn.single_output_score)

In [ ]:
avg_score = nn.single_output_score(X, y)
avg_score

>  ### Average accuracy of individual prediction is less than overall prediction accuracy. Hence, we can conclude that it is better to create one network with N output neurons than N networks each with one output neuron.

## Next Steps
-  ### Generic module for generating optimized network
-  ### Visualize t-SNE for hidden layers

---

## Validation with linear classification methods

In [ ]:
##############  Validation with linear classification methods

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
print(X.shape)
print(y.shape)

tmp = DecisionTreeClassifier(min_samples_leaf=10)
tmp.fit(X_train, y_train)
y_pred = tmp.predict(X_test)
print('test', accuracy_score(y_pred, y_test))
y_pred_train = tmp.predict(X_train)
print('train', accuracy_score(y_pred_train, y_train))

In [ ]:
##############  Validation with linear classification methods

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X_train)
# print(pca.components_)
X_proj = pca.transform(X_test)

f, ax = plt.subplots(2, sharex=True)
f.set_figheight(10)
ax[0].scatter(X_proj[:, 0], X_proj[:, 1], c=np.argmax(y_test, axis=1), alpha=0.9)
ax[1].scatter(X_proj[:, 0], X_proj[:, 1], c=np.argmax(y_pred, axis=1), alpha=0.9)